<b>Install required dependencies and libraries</b>

In [1]:
! pip install lxml html5lib beautifulsoup4

In [2]:
import pandas as pd
import numpy as np

<b>Extract Data Table from URL</b>

In [3]:
#Webpage URL
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Extract Tables
dfs = pd.read_html(url)

#Get the first table
df=dfs[0]

#Extract Columns
dfYTO = df[['Postal Code','Borough', 'Neighbourhood']]
print(dfYTO)        

    Postal Code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
5           M6A        North York   
6           M7A  Downtown Toronto   
7           M8A      Not assigned   
8           M9A         Etobicoke   
9           M1B       Scarborough   
10          M2B      Not assigned   
11          M3B        North York   
12          M4B         East York   
13          M5B  Downtown Toronto   
14          M6B        North York   
15          M7B      Not assigned   
16          M8B      Not assigned   
17          M9B         Etobicoke   
18          M1C       Scarborough   
19          M2C      Not assigned   
20          M3C        North York   
21          M4C         East York   
22          M5C  Downtown Toronto   
23          M6C              York   
24          M7C      Not assigned   
25          M8C      Not assigned   
2

In [4]:
#Check initial entries:
dfYTO.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
#Drop the Borough Rows showing 'Not Assigned'  and save as a new data frame
dfYTO1 = dfYTO[dfYTO.Borough != 'Not assigned']

In [6]:
dfYTO1.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<b>I have used the duplicated function to find out if there was any duplicated Postal Code in the dataframe:</b>

In [7]:
#Command to generate a Dupe Dataframe only for the column Postal Code
PCdupe=dfYTO1['Postal Code'].duplicated()

In [8]:
#Check the PCdupe dataframe: 
PCdupe.head()

2    False
3    False
4    False
5    False
6    False
Name: Postal Code, dtype: bool

In [8]:
#I will also do a value count, only values showing as True would be duplicated: 
PCdupe.value_counts()

False    103
Name: Postal Code, dtype: int64

My finding after the counting and the duplicated function, indicates that there are not duplicated postal codes on the main dataframe dfYTO1. 

<b>Now, I will look for any <i>Not assigned</i> variables in the Neighbourhood column.</b>

In [9]:
#Use .loc function look up for any not assigned variables: 
dfnhood=dfYTO1.loc[dfYTO1['Neighbourhood']=='Not assigned']

In [10]:
dfnhood

,Postal Code,Borough,Neighbourhood


As per the above there are no cells showing as Not Assigned in the current dfYTO1 dataframe.

In [11]:
#shows the shape of my dataframe as it stands now: 
dfYTO1.shape

(103, 3)

After the initial data processing. I have 103 rows and 3 columns.

<b> Now we are going to find the latitude and longitude of the locations to work with Four Square:</b>

When trying to load Geocoder I had some compatibility issues, so my choice was to use the provided file and combine the databases. 

In [12]:
Geo = pd.read_csv("https://cocl.us/Geospatial_data")
print(Geo)

    Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
5           M1J  43.744734 -79.239476
6           M1K  43.727929 -79.262029
7           M1L  43.711112 -79.284577
8           M1M  43.716316 -79.239476
9           M1N  43.692657 -79.264848
10          M1P  43.757410 -79.273304
11          M1R  43.750072 -79.295849
12          M1S  43.794200 -79.262029
13          M1T  43.781638 -79.304302
14          M1V  43.815252 -79.284577
15          M1W  43.799525 -79.318389
16          M1X  43.836125 -79.205636
17          M2H  43.803762 -79.363452
18          M2J  43.778517 -79.346556
19          M2K  43.786947 -79.385975
20          M2L  43.757490 -79.374714
21          M2M  43.789053 -79.408493
22          M2N  43.770120 -79.408493
23          M2P  43.752758 -79.400049
24          M2R  43.782736 -79.442259
25          

In order to make sure the information is added to the correct postal code, I will use an inner join to combine both databases into a new one. My joining variable being the Postal Code.

In [13]:
YTOGeo = pd.merge(left=dfYTO1, right=Geo, left_on='Postal Code', right_on='Postal Code')

In [14]:
YTOGeo.shape

(103, 5)

In [15]:
YTOGeo.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


As per the above, after merging dfYTO and the Geo dataframes, into a new dataframe called YTOGeo, I still landed with 103 rows and 5 columns and when verifying the new dataframe YTOGeo, we have now the Latitude and Longitude for all locations. 

<b>Now, it's time to explore and Cluster the Neighbourhoods in Toronto</b>

As suggested in the IBM/Coursera instructions I have limited my area to work only with the Neighbourhoods in the Borough of Downtown Toronto.

In [16]:
#Code below will download all of the other dependencies we will need. Starting with the Library to handle JSON files:
import json

#Files for Foursquare API
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

#Library to handle requests
import requests 

#transform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules for visuals
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#Map rendering libraries
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

#print 'All libraries Imported at the end of the installation'
print('All libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1h             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2020.6.20          |   py36h9880bd3_2         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

<strong>Note: IBM Watson Studio is the solution I am using for this assignment and it has recently updated to a new version of Python (Python 3.7). I realized when trying to load the above libraries, that they are still not supported by the new version. So, if you face the same problem, I recommend to reverse your file to the previous version Python 3.6.</strong>

<h3><b>Segmenting and Clustering Neighbourhoods in Downtown Toronto</b></h3>

With all libraries installed, I will first cluster the neighbourhoods to show only the ones in Downtown Toronto. 

In [17]:
Downtown_Toronto_data = YTOGeo[YTOGeo['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_Toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [18]:
#I will be using Geopy to get the Geographical Coordinates for Downtown Toronto: 
address = 'Downtown Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [19]:
#Let's visualize the neighbourhoods in Downtown Toronto

#Create map of Downtown Toronto using latitude and longitude values
map_Downtown_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Downtown_Toronto_data['Latitude'], Downtown_Toronto_data['Longitude'], Downtown_Toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Downtown_Toronto)  
    
map_Downtown_Toronto

With the map above plotted, we will now start using Four Square to explore and segment the neighbourhoods. I will conduct a similar analysis of the one taught in the course, as per instructions. 

In [20]:
#Define Four Square Credentials and Version
CLIENT_ID = 'RWXZLBMYPCN4NXJRLTNQNI45MT2V1IOS3JNIYAB1DIFTNNKO' # your Foursquare ID
CLIENT_SECRET = '5OGQBTH2H5QUWTK5CYRPL5BZ2QS5M32X3ZTSKNXHY3RS0EIX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RWXZLBMYPCN4NXJRLTNQNI45MT2V1IOS3JNIYAB1DIFTNNKO
CLIENT_SECRET:5OGQBTH2H5QUWTK5CYRPL5BZ2QS5M32X3ZTSKNXHY3RS0EIX


In [21]:
#Let's explore the first neighbourhood in the dataframe: 
Downtown_Toronto_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

There are two neighbourhoods under the same postal code in the dataframe in our data exploration. They are Regent Park and Harbourfront, meaning they make part of the same borough.

In [22]:
#Let's get the geographical coordinates for Regent Park and Harbourfront: 

neighbourhood_latitude = Downtown_Toronto_data.loc[0, 'Latitude'] # Neighbourhood latitude value
neighbourhood_longitude = Downtown_Toronto_data.loc[0, 'Longitude'] # Neighbourhood longitude value

neighbourhood_name = Downtown_Toronto_data.loc[0, 'Neighbourhood'] # Neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))


Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


Now, let's have a look into the top 100 venues in Regent Park/Harbourfront location within a radius of 500 metres.

In [23]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url #Display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=RWXZLBMYPCN4NXJRLTNQNI45MT2V1IOS3JNIYAB1DIFTNNKO&client_secret=5OGQBTH2H5QUWTK5CYRPL5BZ2QS5M32X3ZTSKNXHY3RS0EIX&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [24]:
#Send a Get Request to examine the results: 
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f9773efb530f20332fdc8d3'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 44,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

Let's now get the Category Types: 

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

It's time to structure the venues into a Panda dataframe.

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


Let's create the function to explore the other neighbourhoods in Downtown Toronto. 

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
         
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
return(nearby_venues)

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980
5,Corktown Common,Park,43.655618,-79.356211
6,Dominion Pub and Kitchen,Pub,43.656919,-79.358967
7,The Distillery Historic District,Historic Site,43.650244,-79.359323
8,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
9,The Extension Room,Gym / Fitness Center,43.653313,-79.359725


It shows that there are only 43 venues in the borough, so we should maybe reduce from a Top 100 to a Top 20 in future instances. I will keep replicating the code from the previous lab from here. 

In [34]:
#Let's check how many venues were returned by FourSquare:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

44 venues were returned by Foursquare.


It shows that there are only 44 venues in the borough, so we should maybe reduce from a Top 100 to a Top 20 in future instances. I will keep replicating the code from the previous lab from here.

Now, let's create a code to do the similar to above to all neighbourhoods in Downtown Toronto: 

In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In the next part of the code, I will create the function to run the above code for each neighbourhood, and will also create a new dataframe named Downtown_Toronto_Venues. 

In [37]:
Downtown_Toronto_venues = getNearbyVenues(names=Downtown_Toronto_data['Neighbourhood'],
                                   latitudes=Downtown_Toronto_data['Latitude'],
                                   longitudes=Downtown_Toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [39]:
#Check the size of the new dataframe
print(Downtown_Toronto_venues.shape)
Downtown_Toronto_venues.head()

(1248, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [40]:
#Check number of venues per neighbourhood:
Downtown_Toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,55,55,55,55,55,55
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,68,68,68,68,68,68
Christie,16,16,16,16,16,16
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [41]:
#Shows how many unique results can be curated from all the returned venues: 
print('There are {} uniques categories.'.format(len(Downtown_Toronto_venues['Venue Category'].unique())))

There are 213 uniques categories.


<b>Let's now analyze each neighbourhood</b>

In [42]:
# one hot encoding
Downtown_Toronto_onehot = pd.get_dummies(Downtown_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Downtown_Toronto_onehot['Neighbourhood'] = Downtown_Toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Downtown_Toronto_onehot.columns[-1]] + list(Downtown_Toronto_onehot.columns[:-1])
Downtown_Toronto_onehot = Downtown_Toronto_onehot[fixed_columns]
Downtown_Toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
#Confirm new size:
Downtown_Toronto_onehot.shape

(1248, 214)

Next, I will group rows by neighborhood and by taking the mean of the frequency of occurrence of each category:

In [45]:
Downtown_Toronto_grouped = Downtown_Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Downtown_Toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.018182,0.000000,0.000000,0.000000,0.00,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.014706,0.000000,0.000000,0.014706,0.00,0.014706
3,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Church and Wellesley,0.013333,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.013333,0.000000,...,0.013333,0.00,0.00,0.00,0.000000,0.000000,0.013333,0.000000,0.00,0.026667
5,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.040000,0.000000,...,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.00,0.000000
6,"First Canadian Place, Underground city",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.040000,0.000000,...,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.00,0.000000
7,"Garden District, Ryerson",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.01,0.00,0.00,0.000000,0.010000,0.010000,0.010000,0.00,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.00,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.054054,0.000000,0.040541,0.013514,0.00,0.000000


Let's confirm the new size:

In [46]:
Downtown_Toronto_grouped.shape

(19, 214)

Let's now print the information for each neighbourhood along with the top 5 venues. 

In [47]:
#Print each neighbourhood along with the top 5 most common venues: 
num_top_venues = 5

for hood in Downtown_Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Downtown_Toronto_grouped[Downtown_Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1  Seafood Restaurant  0.04
2         Cheese Shop  0.04
3        Cocktail Bar  0.04
4            Beer Bar  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0       Airport Lounge  0.12
1      Airport Service  0.12
2      Harbor / Marina  0.06
3              Airport  0.06
4  Rental Car Location  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1                Café  0.06
2  Italian Restaurant  0.04
3      Sandwich Place  0.04
4     Bubble Tea Shop  0.03


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3     Restaurant  0.06
4     Baby Store  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.09
1  Japanese Restaurant  0.05
2     Sushi Restaurant  0.05
3              Gay B

In order to proceed, let's now write the code to put the venues in descending order. 

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Let's create a dataframe to show the top 10 venues for each neighbourhood. 

In [50]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = Downtown_Toronto_grouped['Neighbourhood']

for ind in np.arange(Downtown_Toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Downtown_Toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cheese Shop,Bakery,Cocktail Bar,Seafood Restaurant,Beer Bar,Farmers Market,Restaurant,Breakfast Spot,Shopping Mall
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Harbor / Marina,Bar,Plane,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Coffee Shop
2,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Salad Place,Department Store,Japanese Restaurant,Bubble Tea Shop,Burger Joint,Thai Restaurant
3,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Candy Store,Restaurant,Baby Store,Nightclub,Coffee Shop
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Yoga Studio


<h3><b>Clustering Neighbourhoods</b></h3>

In [51]:
# set number of clusters
kclusters = 5

Downtown_Toronto_grouped_clustering = Downtown_Toronto_grouped.drop('Neighbourhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Downtown_Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 3, 0, 2, 4, 4, 4, 4, 0, 4], dtype=int32)

Now, let's create a new dataframe to include the cluster and top 10 venues: 

In [52]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Downtown_Toronto_merged = Downtown_Toronto_data

# merge Downtown_Toronto_grouped with Downtown_Toronto_data to add latitude/longitude for each neighbourhood
Downtown_Toronto_merged = Downtown_Toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Downtown_Toronto_merged.head() 

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Beer Store,Mexican Restaurant,Spa
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Yoga Studio,Portuguese Restaurant,Italian Restaurant,Smoothie Shop,Beer Bar,Sandwich Place,Distribution Center,Restaurant,Diner
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Hotel,Fast Food Restaurant,Bookstore
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Coffee Shop,Café,Cocktail Bar,Restaurant,Gastropub,Beer Bar,American Restaurant,Gym,Farmers Market,Hotel
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,4,Coffee Shop,Cheese Shop,Bakery,Cocktail Bar,Seafood Restaurant,Beer Bar,Farmers Market,Restaurant,Breakfast Spot,Shopping Mall


Let's visualize the cluster and create a map:

In [53]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Downtown_Toronto_merged['Latitude'], Downtown_Toronto_merged['Longitude'], Downtown_Toronto_merged['Neighbourhood'], Downtown_Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3><b>Examining the Clusters</b></h3>

In [54]:
#Cluster 1
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels'] == 0, Downtown_Toronto_merged.columns[[1] + list(range(5, Downtown_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Beer Store,Mexican Restaurant,Spa
1,Downtown Toronto,0,Coffee Shop,Yoga Studio,Portuguese Restaurant,Italian Restaurant,Smoothie Shop,Beer Bar,Sandwich Place,Distribution Center,Restaurant,Diner
5,Downtown Toronto,0,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Salad Place,Department Store,Japanese Restaurant,Bubble Tea Shop,Burger Joint,Thai Restaurant
8,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Scenic Lookout,Brewery,Restaurant,Park,Plaza


In [55]:
#Cluster 2
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels'] == 1, Downtown_Toronto_merged.columns[[1] + list(range(5, Downtown_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,1,Park,Playground,Trail,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [56]:
#Cluster 3
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels'] == 2, Downtown_Toronto_merged.columns[[1] + list(range(5, Downtown_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,2,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Candy Store,Restaurant,Baby Store,Nightclub,Coffee Shop


In [57]:
#Cluster 4
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels'] == 3, Downtown_Toronto_merged.columns[[1] + list(range(5, Downtown_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,3,Airport Lounge,Airport Service,Harbor / Marina,Bar,Plane,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Coffee Shop


In [58]:
#Cluster 5
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels'] == 4, Downtown_Toronto_merged.columns[[1] + list(range(5, Downtown_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,4,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Hotel,Fast Food Restaurant,Bookstore
3,Downtown Toronto,4,Coffee Shop,Café,Cocktail Bar,Restaurant,Gastropub,Beer Bar,American Restaurant,Gym,Farmers Market,Hotel
4,Downtown Toronto,4,Coffee Shop,Cheese Shop,Bakery,Cocktail Bar,Seafood Restaurant,Beer Bar,Farmers Market,Restaurant,Breakfast Spot,Shopping Mall
7,Downtown Toronto,4,Coffee Shop,Café,Restaurant,Bar,Hotel,Gym,Clothing Store,Thai Restaurant,Office,Steakhouse
9,Downtown Toronto,4,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Salad Place,Japanese Restaurant,Seafood Restaurant,Bakery,Italian Restaurant
10,Downtown Toronto,4,Coffee Shop,Restaurant,Hotel,Café,Gym,American Restaurant,Seafood Restaurant,Japanese Restaurant,Deli / Bodega,Italian Restaurant
11,Downtown Toronto,4,Café,Bookstore,Sandwich Place,Bar,Japanese Restaurant,Bakery,Yoga Studio,Italian Restaurant,Beer Bar,Beer Store
12,Downtown Toronto,4,Café,Vegetarian / Vegan Restaurant,Bar,Coffee Shop,Mexican Restaurant,Vietnamese Restaurant,Gaming Cafe,Dumpling Restaurant,Burger Joint,Pizza Place
15,Downtown Toronto,4,Coffee Shop,Italian Restaurant,Restaurant,Café,Beer Bar,Seafood Restaurant,Pub,Hotel,Japanese Restaurant,Cheese Shop
16,Downtown Toronto,4,Coffee Shop,Restaurant,Café,Pizza Place,Market,Pub,Bakery,Park,Chinese Restaurant,Italian Restaurant


Cluster 5 is the one with the biggest amount of venues. 

<b>Thank you for reviewing my assignment!</b>